# Recursive Neural Networks


In [2]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


## Extract the data

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# path = "/content/drive/Shareddrives/G5/project-4-sentiment-classification/"
path = "./"
train_data = pd.read_csv(path + "train.csv")
test_data = pd.read_csv(path + "test.csv")

print(train_data.shape)
print(test_data.shape)
train_data.head()

(25000, 2)
(25000, 1)


,message,label
0,I saw this movie in NEW York city. I was waiti...,neg
1,This is a German film from 1974 that is someth...,neg
2,I attempted watching this movie twice and even...,neg
3,On his birthday a small boys tells his mother ...,neg
4,"The person who wrote the review ""enough with t...",pos


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeffrey.monja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jeffrey.monja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jeffrey.monja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.isalpha() and token.lower() not in stop_words]
    return ' '.join(tokens)

In [7]:
train_data['tokens'] = train_data['message'].apply(preprocess_text)
test_data['tokens'] = test_data['message'].apply(preprocess_text)

In [8]:
train_data.head()

,message,label,tokens
0,I saw this movie in NEW York city. I was waiti...,neg,saw movie new york city waiting bus next morni...
1,This is a German film from 1974 that is someth...,neg,german film something woman come castle beyond...
2,I attempted watching this movie twice and even...,neg,attempted watching movie twice even fast forwa...
3,On his birthday a small boys tells his mother ...,neg,birthday small boy tell mother son want go hom...
4,"The person who wrote the review ""enough with t...",pos,person wrote review enough sweating spitting a...


In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()  
train_data['label'] = label_encoder.fit_transform(train_data['label'])

In [10]:
from gensim.models import Word2Vec

sentences = train_data['tokens'].apply(lambda x: x.split()).to_list()
word2vec = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, sg=1)

sentences_test = test_data['tokens'].apply(lambda x: x.split()).to_list()
word2vec_test = Word2Vec(sentences=sentences_test, vector_size=100, window=5, min_count=1, sg=1)

In [11]:
word2vec.wv.most_similar("like", topn=5)

[('creepier', 0.8047286868095398),
 ('freaked', 0.7866225242614746),
 ('scrat', 0.7834646701812744),
 ('preteen', 0.7825940251350403),
 ('ch', 0.7808871269226074)]

In [12]:
def sentence_to_vectors(sentence, model, vector_size=100):
    vectors = []
    for word in sentence.split():
        if word in model.wv:
            vectors.append(model.wv[word])
        else:
            vectors.append([0] * vector_size)
    return np.array(vectors)

In [13]:
train_data['vectors'] = train_data['tokens'].apply(lambda x: sentence_to_vectors(x, word2vec))
test_data['vectors'] = test_data['tokens'].apply(lambda x: sentence_to_vectors(x, word2vec_test))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2), lowercase=True)
train_data['tfidf'] = list(vectorizer.fit_transform(train_data['tokens']).toarray())
test_data['tfidf'] = list(vectorizer.transform(test_data['tokens']).toarray())

In [15]:
max_len = max(train_data['tokens'].apply(lambda text: len(text.split())))
mean_len = int(round(np.mean(train_data['tokens'].apply(lambda text: len(text.split())))))

print(f'Max length: {max_len}')
print(f'Mean length: {mean_len}')

Max length: 1421
Mean length: 119


In [16]:
def pad_sentences(vectors, max_len, vector_size=100):
    if len(vectors) > max_len:
        vectors = vectors[:max_len]
    else:
        padding = np.zeros((max_len - len(vectors), vector_size))
        vectors = np.vstack([vectors, padding])
    return torch.tensor(vectors, dtype=torch.float32)

In [17]:
train_data['tensor'] = train_data['vectors'].apply(lambda x: pad_sentences(x, mean_len, word2vec.vector_size))
test_data['tensor'] = test_data['vectors'].apply(lambda x: pad_sentences(x, mean_len, word2vec_test.vector_size))

In [18]:
train_data.head()

,message,label,tokens,vectors,tfidf,tensor
0,I saw this movie in NEW York city. I was waiti...,0,saw movie new york city waiting bus next morni...,"[[-0.21930268, 0.34450474, -0.6291782, -0.4775...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[tensor(-0.2193), tensor(0.3445), tensor(-0.6..."
1,This is a German film from 1974 that is someth...,0,german film something woman come castle beyond...,"[[-0.33192742, 0.4755079, -0.21171649, -0.3054...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[tensor(-0.3319), tensor(0.4755), tensor(-0.2..."
2,I attempted watching this movie twice and even...,0,attempted watching movie twice even fast forwa...,"[[-0.24511866, 0.021486083, 0.41561908, 0.0468...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[tensor(-0.2451), tensor(0.0215), tensor(0.41..."
3,On his birthday a small boys tells his mother ...,0,birthday small boy tell mother son want go hom...,"[[0.18536536, 0.57395875, -0.559319, -0.181436...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[tensor(0.1854), tensor(0.5740), tensor(-0.55..."
4,"The person who wrote the review ""enough with t...",1,person wrote review enough sweating spitting a...,"[[-0.03453762, 0.2127353, 0.123608805, 0.44025...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[tensor(-0.0345), tensor(0.2127), tensor(0.12..."


In [19]:
X = torch.stack(train_data['tensor'].tolist())
y = torch.tensor(train_data['label'].values)

print(X.shape)
print(y.shape)

torch.Size([25000, 119, 100])
torch.Size([25000])


In [20]:
from torch.utils.data import DataLoader, TensorDataset, random_split

dataset = TensorDataset(X, y)

train_size = int(0.8 * len(dataset))  # 80% para entrenamiento
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(len(train_loader)) 
print(len(test_loader)) 

625
157


## Models

In [21]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)  
        hidden = hidden[-1]
        out = self.fc(hidden)
        return out

# class LSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, num_layers=1):
#         super(LSTM, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)

#     def forward(self, window):
#         _, (h_out, _) = self.lstm(window)
#         h_out = h_out.view(-1, self.hidden_size)
#         out = self.fc(h_out)
#         return out

In [22]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, hidden = self.rnn(x) 
        hidden = hidden[-1] 
        out = self.fc(hidden)
        return out

In [23]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(GRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, hidden = self.gru(x)
        hidden = hidden[-1]
        out = self.fc(hidden)
        return out

## Training Function
This function is designed to train all three models efficiently and streamline the process for the user.

In [24]:
def train(model, optimizer, loss_f, num_epochs, train_loader):
    losses = []
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for inputs, labels in train_loader:
            labels = labels.long()
            outputs = model(inputs)
            loss = loss_f(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")

## Hyperparameters
Define the model's hyperparameters

In [25]:
num_epochs = 10
learning_rate = 0.001

input_size = 100 # word2vec.vector_size
hidden_size = 128
output_size = 2
num_layers = 1  

## Create and train the models

### LSTM

In [ ]:
lstm = LSTM(input_size, hidden_size, output_size, num_layers)
loss_function_LSTM = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

train(lstm, optimizer, loss_function_LSTM, num_epochs, train_loader)

In [27]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return accuracy

In [28]:
# Función de búsqueda en cuadrícula de hiperparámetros
def grid_search(param_grid):
    best_accuracy = 0
    best_params = {}

    # Iterar sobre todas las combinaciones posibles de hiperparámetros
    for hidden_size in param_grid['hidden_size']:
        for num_layers in param_grid['num_layers']:
            for learning_rate in param_grid['learning_rate']:
                for batch_size in param_grid['batch_size']:
                    for num_epochs in param_grid['num_epochs']:

                        # Crear DataLoader con el tamaño de batch actual
                        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

                        # Inicializar el modelo LSTM con los parámetros actuales
                        model = LSTM(input_size=100, hidden_size=hidden_size, output_size=2, num_layers=num_layers)

                        # Inicializar la función de pérdida y el optimizador
                        loss_function_LSTM = torch.nn.CrossEntropyLoss()
                        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

                        # Entrenar el modelo
                        print(f"Entrenando con hidden_size={hidden_size}, num_layers={num_layers}, "
                              f"learning_rate={learning_rate}, batch_size={batch_size}, num_epochs={num_epochs}")
                        train(model, optimizer, loss_function_LSTM, num_epochs, train_loader)

                        # Evaluar el modelo
                        accuracy = evaluate(model, test_loader)
                        print(f"Accuracy con hidden_size={hidden_size}, num_layers={num_layers}, "
                              f"learning_rate={learning_rate}, batch_size={batch_size}, num_epochs={num_epochs}: {accuracy * 100:.2f}%")

                        # Guardar la mejor configuración
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_params = {
                                'hidden_size': hidden_size,
                                'num_layers': num_layers,
                                'learning_rate': learning_rate,
                                'batch_size': batch_size,
                                'num_epochs': num_epochs
                            }
    
    return best_accuracy, best_params

In [30]:
# Parámetros para la búsqueda en cuadrícula
param_grid = {
    'hidden_size': [64, 128, 256],           # Diferentes tamaños de la capa oculta
    'num_layers': [1, 2, 3],                 # Número de capas LSTM
    'learning_rate': [0.001, 0.0005, 0.0001],# Diferentes tasas de aprendizaje
    'batch_size': [32, 64, 128],             # Tamaño del batch
    'num_epochs': [10, 20, 30]                   # Número de épocas para entrenar
}

# Realizar búsqueda en cuadrícula
best_accuracy, best_params = grid_search(param_grid)

# Imprimir los mejores parámetros y la precisión
print(f"\nMejores parámetros: {best_params}")
print(f"Mejor Accuracy: {best_accuracy * 100:.2f}%")

Entrenando con hidden_size=64, num_layers=1, learning_rate=0.001, batch_size=32, num_epochs=10
Epoch 1/10, Loss: 0.6858070622444152
Epoch 2/10, Loss: 0.6863924999237061
Epoch 3/10, Loss: 0.6765423334121704
Epoch 4/10, Loss: 0.6678766390323639
Epoch 5/10, Loss: 0.6587735070228576
Epoch 6/10, Loss: 0.6174238119602203
Epoch 7/10, Loss: 0.40239698531627655
Epoch 8/10, Loss: 0.3501253200292587
Epoch 9/10, Loss: 0.33415063428878783
Epoch 10/10, Loss: 0.3224551130056381
Accuracy con hidden_size=64, num_layers=1, learning_rate=0.001, batch_size=32, num_epochs=10: 85.02%
Entrenando con hidden_size=64, num_layers=1, learning_rate=0.001, batch_size=32, num_epochs=20
Epoch 1/20, Loss: 0.6877557843208313
Epoch 2/20, Loss: 0.6663573081970214
Epoch 3/20, Loss: 0.6581019725322723
Epoch 4/20, Loss: 0.6881966814041137
Epoch 5/20, Loss: 0.6669482594490052
Epoch 6/20, Loss: 0.6890221713066101
Epoch 7/20, Loss: 0.5935684298038483
Epoch 8/20, Loss: 0.3649332551121712
Epoch 9/20, Loss: 0.331450732254982
Epoc

KeyboardInterrupt: 

### RNN

In [49]:
rnn = RNN(input_size, hidden_size, output_size, num_layers)
loss_function_RNN = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
train(rnn, optimizer, loss_function_RNN, num_epochs, train_loader)

Epoch 1/30, Loss: 0.6794653319835663
Epoch 2/30, Loss: 0.6824531795978546
Epoch 3/30, Loss: 0.6913336690902709
Epoch 4/30, Loss: 0.6706020780563354
Epoch 5/30, Loss: 0.6765170733451843
Epoch 6/30, Loss: 0.6700386498451233
Epoch 7/30, Loss: 0.6504225824832917
Epoch 8/30, Loss: 0.6611987741470337
Epoch 9/30, Loss: 0.6874834663391113
Epoch 10/30, Loss: 0.6852647385597229
Epoch 11/30, Loss: 0.6820053078651428
Epoch 12/30, Loss: 0.6822574971199036
Epoch 13/30, Loss: 0.6760899051666259
Epoch 14/30, Loss: 0.6828932273864746
Epoch 15/30, Loss: 0.6846454354286193
Epoch 16/30, Loss: 0.6752873534202576
Epoch 17/30, Loss: 0.6712932606697083
Epoch 18/30, Loss: 0.6513241497516632
Epoch 19/30, Loss: 0.6773011771202088
Epoch 20/30, Loss: 0.6637817116737366
Epoch 21/30, Loss: 0.6677394290924072
Epoch 22/30, Loss: 0.6562525920391082
Epoch 23/30, Loss: 0.6268605331897735
Epoch 24/30, Loss: 0.6468837797641754
Epoch 25/30, Loss: 0.5980380709171296
Epoch 26/30, Loss: 0.6667880938529969
Epoch 27/30, Loss: 0.

### GRU

In [ ]:
gru = GRU(input_size, hidden_size, output_size, num_layers)
loss_function_GRU = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=learning_rate)
train(gru, optimizer, loss_function_GRU, num_epochs, train_loader)